# Zero Filled

This is a simple baseline, a demonstration on a submission.

In [14]:
import os
import fastmri
import h5py
from fastmri.data import transforms as T
from fastmri.data.mri_data import et_query
from tqdm import tqdm
from pathlib import Path

import xml.etree.ElementTree as etree

We set data source directory and output directory, together with which challenge we want to submit. Here we only choose ` knee_sc`.

In [30]:
DATA = 'Datasets'
PARTITION = 'Val'
challenge = 'knee_sc'

data_dir = Path(os.path.join(DATA, PARTITION))

out_dir = Path('reconstruction')

reconstructions = {}
print(list(data_dir.glob("*.h5")))

for fname in tqdm(list(data_dir.glob("*.h5"))):
    with h5py.File(fname, "r") as hf:
        et_root = etree.fromstring(hf["ismrmrd_header"][()])
        masked_kspace = T.to_tensor(hf["kspace"][()])

        # extract target image width, height from ismrmrd header
        enc = ["encoding", "encodedSpace", "matrixSize"]
        crop_size = (
            int(et_query(et_root, enc + ["x"])),
            int(et_query(et_root, enc + ["y"]))
        )

        # inverse Fourier Transform to get zero filled solution
        image = fastmri.ifft2c(masked_kspace)

        # check for FLAIR 203
        if image.shape[-2] < crop_size[1]:
            cro_size (image.shape[-2], image.shape[-2])

        # crop input image
        image = T.complex_center_crop(image, crop_size)

        # absolute value
        image = fastmri.complex_abs(image)

        # apply root-sum-of-squares if multicoil data
        if challenge == 'multicoil':
            image = fastmri.rss(image, dim=1)

        reconstructions[fname.name] = image

fastmri.save_reconstructions(reconstructions, out_dir)

[PosixPath('Datasets/Val/file1000000.h5')]


100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.42it/s]
